[Reference](https://blog.bbelyakov.com/stock-price-forecasting-with-lag-llama-transformers-2d6ada22a088)

In [2]:
!git clone https://github.com/time-series-foundation-models/lag-llama

fatal: destination path 'lag-llama' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [4]:
cd /content/lag-llama

/content/lag-llama


In [5]:
!ls

configs  gluon_utils  lag_llama  README.md	   run.py   utils
data	 images       LICENSE	 requirements.txt  scripts


In [6]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 48.9 MB/s eta 0:00:00
ERROR: pip's dependency reso

In [7]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llaz

lag-llama.ckpt: 100% 29.5M/29.5M [00:00<00:00, 85.4MB/s]
Download complete. Moving file to /content/lag-llaz/lag-llama.ckpt
/content/lag-llaz/lag-llama.ckpt


In [9]:
from itertools import islice
import matplotlib.dates as mdates
import pandas as pd
import torch
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation import Evaluator, make_evaluation_predictions
from lag_llama.gluon.estimator import LagLlamaEstimator
from matplotlib import pyplot as plt

In [10]:
def _get_lag_llama_predictions(dataset, prediction_length, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=torch.device("cuda:0"))
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,

        # pretrained length
        context_length=32,

        # estimator arguments
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        batch_size=1,
        num_parallel_samples=100,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset, predictor=predictor, num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [11]:
# using the latest version available
!pip install yfinance==0.2.37

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.40
    Uninstalling yfinance-0.2.40:
      Successfully uninstalled yfinance-0.2.40


In [12]:
import yfinance as yf


# adjust the tickers, period, and frequency as needed
stock_prices = (
    yf.Tickers("aapl amd amzn crm goog meta msft nvda tsla")
    .history(period="max", start="2013-01-01")
    .Close
    .resample('1d')
    .ffill()
)
stock_prices

[*********************100%%**********************]  9 of 9 completed


Ticker,AAPL,AMD,AMZN,CRM,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,,,
2013-01-02,16.725037,2.530000,12.865500,42.736675,18.013729,27.970324,22.492283,2.935841,2.357333
2013-01-03,16.513926,2.490000,12.924000,42.122478,18.024191,27.740568,22.190975,2.938149,2.318000
2013-01-04,16.053934,2.590000,12.957500,42.347183,18.380356,28.729517,21.775658,3.035087,2.293333
2013-01-05,16.053934,2.590000,12.957500,42.347183,18.380356,28.729517,21.775658,3.035087,2.293333
2013-01-06,16.053934,2.590000,12.957500,42.347183,18.380356,28.729517,21.775658,3.035087,2.293333
...,...,...,...,...,...,...,...,...,...
2024-05-25,189.979996,166.360001,180.750000,272.290009,176.330002,478.220001,430.160004,1064.689941,179.240005
2024-05-26,189.979996,166.360001,180.750000,272.290009,176.330002,478.220001,430.160004,1064.689941,179.240005
2024-05-27,189.979996,166.360001,180.750000,272.290009,176.330002,478.220001,430.160004,1064.689941,179.240005


In [13]:
import yfinance as yf


# adjust the tickers, period, and frequency as needed
stock_prices = (
    yf.Tickers("aapl amd amzn crm goog meta msft nvda tsla")
    .history(period="max", start="2013-01-01")
    .Close
    .resample('1d')
    .ffill()
)
stock_prices

[*********************100%%**********************]  9 of 9 completed


Ticker,AAPL,AMD,AMZN,CRM,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,,,
2013-01-02,16.725037,2.530000,12.865500,42.736675,18.013729,27.970324,22.492283,2.935841,2.357333
2013-01-03,16.513926,2.490000,12.924000,42.122478,18.024191,27.740568,22.190973,2.938148,2.318000
2013-01-04,16.053944,2.590000,12.957500,42.347183,18.380356,28.729517,21.775652,3.035086,2.293333
2013-01-05,16.053944,2.590000,12.957500,42.347183,18.380356,28.729517,21.775652,3.035086,2.293333
2013-01-06,16.053944,2.590000,12.957500,42.347183,18.380356,28.729517,21.775652,3.035086,2.293333
...,...,...,...,...,...,...,...,...,...
2024-05-25,189.979996,166.360001,180.750000,272.290009,176.330002,478.220001,430.160004,1064.689941,179.240005
2024-05-26,189.979996,166.360001,180.750000,272.290009,176.330002,478.220001,430.160004,1064.689941,179.240005
2024-05-27,189.979996,166.360001,180.750000,272.290009,176.330002,478.220001,430.160004,1064.689941,179.240005


In [ ]:
def _get_lag_llama_dataset(dataset):
    # avoid mutations
    dataset = dataset.copy()

    # convert numerical columns to `float32`
    for col in dataset.columns:
        if dataset[col].dtype != "object" and not pd.api.types.is_string_dtype(
            dataset[col]
        ):
            dataset[col] = dataset[col].astype("float32")

    # create a `PandasDataset`
    backtest_dataset = PandasDataset(dict(dataset))
    return backtest_dataset


backtest_dataset = _get_lag_llama_dataset(dataset=stock_returns)
prediction_length = 60  # prediction length
num_samples = 1060 # sampled from the distribution for each timestep

In [ ]:
forecasts, tss = _get_lag_llama_predictions(
    backtest_dataset, prediction_length, num_samples
)

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter("%b, %d")
plt.rcParams.update({"font.size": 15})

# iterate through the series, plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx + 1)

    plt.plot(
        ts[(-4 * prediction_length):].to_timestamp(),
        label="target",
    )
    forecast.plot(color="g")
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter("%b, %d")
plt.rcParams.update({"font.size": 15})

# iterate through the series, plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx + 1)

    plt.plot(
        ts[(-4 * prediction_length):].to_timestamp(),
        label="target",
    )
    forecast.plot(color="g")
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
# e.g., using the first 5 years of data
# we will then test on the rest of the dataset
train_dataset = _get_lag_llama_dataset(stock_returns.iloc[:252*5])
predictor = estimator.train(
    train_dataset,
    cache_data=True,
    shuffle_buffer_length=1000,
)

In [ ]:
def _get_lag_llama_predictions(
    dataset, prediction_length, num_samples=100, predictor=None,
):
    ckpt = torch.load(
        "lag-llama.ckpt", map_location=torch.device("cuda:0")
    )
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,

        # pretrained length
        context_length=32,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        batch_size=1,
        num_parallel_samples=100,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = (
        predictor
        if predictor is not None
        else estimator.create_predictor(transformation, lightning_module)
    )

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset, predictor=predictor, num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss